In [2]:
%pwd

'/Users/sidharthgoel/Desktop/deep_learning_disease_detection/Chickdis/research'

In [3]:
import os
import urllib.request as request
import zipfile
from Chickdis import logger
from Chickdis.utils.common import get_size

In [4]:
os.chdir("/Users/sidharthgoel/Desktop/deep_learning_disease_detection/Chickdis")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from Chickdis.constants import *
from Chickdis.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.root_dir):
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Created directory: {self.config.root_dir}")

        logger.info(f"Downloading data from {self.config.source_URL} to {self.config.local_data_file}")
        request.urlretrieve(self.config.source_URL, self.config.local_data_file)
        logger.info(f"Downloaded file size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        logger.info(f"Extracting zip file: {self.config.local_data_file} to {self.config.unzip_dir}")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info("Extraction completed.")

    def initiate_data_ingestion(self):
        self.download_data()
        self.extract_zip_file()
        
    

In [11]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)
data_ingestion.initiate_data_ingestion()

ValueError: The file at config/config.yaml is empty.